In [43]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [44]:
jack = Customer('Park', 100)
jack

Customer(name='Park', fidelity=100)

In [45]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

In [46]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion #할인 객체

    def total(self):
        """ __total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self,'__tatal'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total : {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [47]:
class Promotion(ABC): # strategy : abstract base class
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass


In [48]:
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0
        

In [49]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('Watermellon', 5, 5.0)]

In [50]:
Order(joe, cart, FidelityPromo())

<Order total : 42.00 due: 42.00>

In [51]:
Order(ann, cart, FidelityPromo())

<Order total : 42.00 due: 39.90>

In [52]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [53]:
Order(joe, banana_cart, BulkItemPromo())

<Order total : 30.00 due: 28.50>

In [54]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]


In [55]:
Order(joe, long_order, LargeOrderPromo())

<Order total : 10.00 due: 9.30>

In [56]:
Order(joe, cart, LargeOrderPromo())

<Order total : 42.00 due: 42.00>

In [71]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total : {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    

In [72]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0 

In [73]:
Order(joe, cart, fidelity_promo)

<Order total : 42.00 due: 42.00>

In [74]:
Order(ann, cart, fidelity_promo)

<Order total : 42.00 due: 39.90>

In [75]:
Order(joe, banana_cart, bulk_item_promo)

<Order total : 30.00 due: 28.50>

In [76]:
Order(ann, long_order, large_order_promo)


<Order total : 10.00 due: 9.30>

In [77]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# 함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [78]:
Order(joe, long_order, best_promo)


<Order total : 10.00 due: 9.30>

In [79]:
Order(joe, banana_cart, best_promo)

<Order total : 30.00 due: 28.50>

In [81]:
Order(ann, cart, best_promo)

<Order total : 42.00 due: 39.90>

In [82]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractclassmethod', 'namedtuple', 'Customer', '_i2', 'jack', '_2', '_i3', 'LineItem', '_i4', 'Order', '_i5', '_i6', '_i7', 'Promotion', '_i8', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i9', 'joe', 'ann', 'cart', '_i10', '_10', '_i11', '_i12', '_12', '_i13', '_i14', '_i15', '_i16', '_i17', '_17', '_i18', '_18', '_i19', 'banana_cart', '_i20', '_20', '_i21', 'long_order', '_i22', '_22', '_i23', '_23', '_i24', '_i25', '_i26', '_i27', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i28', '_28', '_i29', '_29', '_i30', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35', '_35', '_i36', '_i37', '_i38', '_38', '_i39', '_39', '_i40', '_40', '_i41', 'abstractmethod', '_i42', '_42', '_i43', '_i44', '_44', 

In [83]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [84]:
promos = [globals() [name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [85]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [86]:
@deco
def target():
    print("running target()")

target()

running inner()


In [87]:
target

<function __main__.deco.<locals>.inner()>

In [91]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry -->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()


running register(<function f1 at 0x000001D30528C220>)
running register(<function f2 at 0x000001D3052F4860>)
running main()
registry --> [<function f1 at 0x000001D30528C220>, <function f2 at 0x000001D3052F4860>]
running f1()
running f2()
running f3()


In [92]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0 

@promotion
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [93]:
def f1(a):
    print(a)
    print(b)
f1(3)

3


NameError: name 'b' is not defined

In [94]:
b = 6
f1(3)

3
6


In [95]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [96]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [98]:
b

9

In [99]:
f3(3)

3
9


In [100]:
b = 30
f3(3)

3
30


In [101]:
class Averager():
    def __init__ (self):
        self.series = []

    def __call__ (self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [102]:
avg = Averager()
avg(10)

10.0

In [103]:
avg(11)

10.5

In [104]:
avg(12)

11.0

In [105]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [106]:
avg = make_averager()
avg(10)

10.0

In [107]:
avg(11)

10.5

In [108]:
avg(12)

11.0

In [111]:
avg.__code__.co_varnames

('new_value', 'total')

In [112]:
avg.__code__.co_freevars

('series',)

In [113]:
avg.__closure__

(<cell at 0x000001D3055168C0: list object at 0x000001D3053FAE40>,)

In [114]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [120]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [121]:
avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [122]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager


In [123]:
avg = make_averager()
avg(10)

10.0

In [135]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [136]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorail(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12305120s] snooze(0.123) -> None
**************************************** Calling factorail(6)
[0.00000040s] factorial(1) -> 1
[0.00000860s] factorial(2) -> 2
[0.00001440s] factorial(3) -> 6
[0.00002040s] factorial(4) -> 24
[0.00002550s] factorial(5) -> 120
[0.00003160s] factorial(6) -> 720
6! = 720


In [137]:
factorial.__name__

'clocked'

In [138]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(','.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s = %r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [139]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorail(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12402749s] snooze(0.123) -> None
**************************************** Calling factorail(6)
[0.00000000s] factorial(1) -> 1
[0.00000000s] factorial(2) -> 2
[0.00000000s] factorial(3) -> 6
[0.00000000s] factorial(4) -> 24
[0.00000000s] factorial(5) -> 120
[0.00000000s] factorial(6) -> 720
6! = 720


In [140]:
factorial.__name__

'factorial'